In [32]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Mediator Pattern

The Mediator Pattern is a behavioral design pattern that facilitates communication between multiple objects (colleagues) by introducing a mediator object. Instead of objects interacting direcly, they communicate through the mediator, promoting loose coupling.

## Use Case:

In an IA agents framework:
* Agents and tools might need to interact dynamically. For example, an agent may need to use a search tool, a database connector, or another agent to fulfill a task. Instead of each agent/tool being aware of others, a mediator can orchestrate communication.
* Multi-agent orchestration, where the mediator directs interactions between agents with specific roles.

#### Python Example:

In [33]:
class Mediator:
    def notify(self, sender, event):
        raise NotImplementedError
    
class TaskMediator(Mediator):
    def __init__(self):
        self.agents = []

    def register_agent(self, agent):
        self.agents.append(agent)
        agent.set_mediator(self)

    def notify(self, sender, event):
        if event == "task_completed":
            print(f"{sender.name} completed a task. Notifying others.")
            for agent in self.agents:
                if agent != sender:
                    agent.receive_notification(sender.name)

class Agent:
    def __init__(self, name):
        self.name = name
        self.mediator = None
    
    def set_mediator(self, mediator):
        self.mediator = mediator

    def perform_task(self):
        print(f"{self.name} is performing a task.")
        self.mediator.notify(self, "task_completed")

    def receive_notification(self, sender_name):
        print(f"{self.name} received notification from {sender_name}.")

mediator = TaskMediator()

agent1 = Agent('Agent 1')
agent2 = Agent('Agent 2')
agent3 = Agent('Agent 3')

mediator.register_agent(agent1)
mediator.register_agent(agent2)
mediator.register_agent(agent3)

### Output

In [34]:
agent1.perform_task()

Agent 1 is performing a task.
Agent 1 completed a task. Notifying others.
Agent 2 received notification from Agent 1.
Agent 3 received notification from Agent 1.


In [36]:
# Mediator Interface
class Mediator:
    def notify(self, sender, event):
        raise NotImplementedError

# Concrete Mediator
class AgentMediator(Mediator):
    def __init__(self):
        self.agents = {}

    def register_agent(self, role, agent):
        self.agents[role] = agent
        agent.set_mediator(self)

    def notify(self, sender, event, data=None):
        if event == "text_extracted":
            print(f"[Mediator] Text extracted by {sender}. Sending to SummarizerAgent.")
            self.agents["summarizer"].process(data)
        elif event == "summary_generated":
            print(f"[Mediator] Summary generated by {sender}. Sending to TranslatorAgent.")
            self.agents["translator"].process(data)
        elif event == "translation_completed":
            print(f"[Mediator] Translation completed by {sender}. Delivering final result.")
            print(f"Final Translated Summary: {data}")

# Base Agent
class Agent:
    def __init__(self, name):
        self.name = name
        self.mediator = None

    def set_mediator(self, mediator):
        self.mediator = mediator

# ReaderAgent: Extracts text
class ReaderAgent(Agent):
    def process(self, document):
        print(f"[{self.name}] Reading and extracting text from the document.")
        extracted_text = "This is the extracted text from the document."
        self.mediator.notify(self.name, "text_extracted", extracted_text)

# SummarizerAgent: Summarizes text
class SummarizerAgent(Agent):
    def process(self, text):
        print(f"[{self.name}] Summarizing the extracted text.")
        summary = "This is a summary of the document."
        self.mediator.notify(self.name, "summary_generated", summary)

# TranslatorAgent: Translates text
class TranslatorAgent(Agent):
    def process(self, summary):
        print(f"[{self.name}] Translating the summary into Spanish.")
        translation = "Este es un resumen del documento."
        self.mediator.notify(self.name, "translation_completed", translation)

# Usage
mediator = AgentMediator()

reader = ReaderAgent("ReaderAgent")
summarizer = SummarizerAgent("SummarizerAgent")
translator = TranslatorAgent("TranslatorAgent")

mediator.register_agent("reader", reader)
mediator.register_agent("summarizer", summarizer)
mediator.register_agent("translator", translator)

# Start the process
reader.process("sample_document.txt")


[ReaderAgent] Reading and extracting text from the document.
[Mediator] Text extracted by ReaderAgent. Sending to SummarizerAgent.
[SummarizerAgent] Summarizing the extracted text.
[Mediator] Summary generated by SummarizerAgent. Sending to TranslatorAgent.
[TranslatorAgent] Translating the summary into Spanish.
[Mediator] Translation completed by TranslatorAgent. Delivering final result.
Final Translated Summary: Este es un resumen del documento.


#### References:

1. https://refactoring.guru/design-patterns/mediator

2. https://refactoring.guru/design-patterns/mediator/python/example#example-0